In [1]:
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# # Create a sample DataFrame with categorical columns
# data = {'Color': ['Red', 'Green', 'Blue', 'Red', 'Green'],
#         'Size': ['Small', 'Medium', 'Large', 'Small', 'Large']}
# df = pd.DataFrame(data)

# # Initialize LabelEncoder
# label_encoder = LabelEncoder()

# # Iterate over columns and encode categorical values
# for column in df.columns:
#     if df[column].dtype == 'object':  # Check if column contains categorical data
#         df[column] = label_encoder.fit_transform(df[column])

# print(df)


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle



In [14]:



# Load the dataset
data = pd.read_csv("dataset.csv")
# Create clusters of diseases using K-Means (as shown in previous steps)
# ...
# Create a dictionary to map diseases to symptom lists
disease_symptoms = {}
for index, row in data.iterrows():
    disease = row['disease']
    symptoms = [symptom.strip() for symptom in row.values[1:] if isinstance(symptom, str)]
    
    if disease in disease_symptoms:
        disease_symptoms[disease].extend(symptoms)
    else:
        disease_symptoms[disease] = symptoms

# Convert the dictionary values (symptoms) to strings
for disease in disease_symptoms:
    disease_symptoms[disease] = ' '.join(disease_symptoms[disease])

# Create a list of symptom strings
symptom_strings = list(disease_symptoms.values())


# Apply K-Means clustering
num_clusters = 41  # You can adjust the number of clusters based on your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Assign cluster labels to diseases
# disease_clusters = {}
# for disease, symptoms in disease_symptoms.items():
#     symptom_vector = vectorizer.transform([symptoms])
#     cluster_label = kmeans.predict(symptom_vector)[0]
    
#     if cluster_label in disease_clusters:
#         disease_clusters[cluster_label].append(disease)
#     else:
#         disease_clusters[cluster_label] = [disease]

# # Print the clusters and the diseases in each cluster
# for cluster_label, diseases in disease_clusters.items():
#     print(f"Cluster {cluster_label + 1}:")
#     for disease in diseases:
#         print(f" - {disease}")




# Create cluster features for each disease
# for cluster_label in disease_clusters:
#     data[f'Cluster_{cluster_label + 1}'] = data['disease'].apply(lambda x: int(x in disease_clusters[cluster_label]))

symp=data.drop('disease',axis=1)
# Create binary columns for each symptom
for column in symp.columns:
    symp[column] = symp[column].apply(lambda x: 1 if x == 1 else 0)

print(symp)


# label_encoder = LabelEncoder()
# encoded_symptoms = label_encoder.fit_transform(data.drop(['disease'], axis=1))

# X = symp.reshape(-1, 1)  # Features
# y = data['disease']  # Target


      Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  \
0              0           0           0           0           0           0   
1              0           0           0           0           0           0   
2              0           0           0           0           0           0   
3              0           0           0           0           0           0   
4              0           0           0           0           0           0   
...          ...         ...         ...         ...         ...         ...   
4915           0           0           0           0           0           0   
4916           0           0           0           0           0           0   
4917           0           0           0           0           0           0   
4918           0           0           0           0           0           0   
4919           0           0           0           0           0           0   

      Unnamed: 7  Unnamed: 8  Unnamed: 

KeyError: "['prognosis'] not found in axis"

In [9]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(symp, data['disease'], test_size=0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# num_classes = len(np.unique(y_train))
# print(num_classes)
# if num_classes < 2:
#     raise ValueError("Training data should contain samples from at least 2 classes.")


# Build and train a logistic regression model
logreg_model = LogisticRegression(max_iter=1000, multi_class='ovr')  # You can use other classifiers too
logreg_model.fit(X_train, y_train)

# Make predictions
y_pred = logreg_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.0152


In [10]:
with open("logreg_model.pkl", "wb") as model_file:
    pickle.dump(logreg_model, model_file)

In [11]:
def predict_disease(symptoms):
    # Convert the input symptom names into a binary format using the column names
    input_data = pd.DataFrame([[1 if col in symptoms else 0 for col in X.columns]], columns=X.columns)
    # print(input_data)
    # Standardize the input data using the same scaler as before
    # input_data_scaled = scaler.fit_transform(input_data)
    # print(input_data_scaled)

    # Make predictions using individual models
    rf_prediction = logreg_model.predict(input_data)
    
    # Create an ensemble prediction by majority voting
    
    # Decode the predicted label using the label encoder
    # predicted_disease = [le.inverse_transform([rf_prediction]),le.inverse_transform([xgboost_prediction]),le.inverse_transform([ensemble_prediction])]
    predicted_disease = {
        'Logreg' : disease_encoder.inverse_transform([rf_prediction]),
    }
    
    return predicted_disease

In [12]:
predict_disease(["chills","nausea"])

AttributeError: 'csr_matrix' object has no attribute 'columns'

In [7]:
# # Sample Rasa chatbot code (simplified)
# # Install Rasa: pip install rasa

# # Define the Rasa chatbot
# domain = """
# intents:
# - get_symptoms

# responses:
#   utter_ask_name:
#   - text: "Hello! What's your name?"

#   utter_ask_symptoms:
#   - text: "Hi {name}! Please provide your symptoms, separated by commas."

#   utter_show_prediction:
#   - text: "Based on the symptoms you provided, the predicted disease is {prediction}."

# actions:
# - action_predict_disease
# """

# import random
# from rasa_sdk import Action, Tracker
# from rasa_sdk.executor import CollectingDispatcher

# # Define the symptom prediction action
# class ActionPredictDisease(Action):
#     def name(self) -> str:
#         return "action_predict_disease"
#     # Sample symptom prediction function
# def predict_disease(symptoms):
#     # Preprocess the symptoms (convert to binary encoding or any other suitable format)
#     preprocessed_symptoms = preprocess_symptoms(symptoms)

#     # Make predictions using your model
#     predicted_disease = your_model.predict(preprocessed_symptoms)

#     # Return the predicted disease
#     return predicted_disease

#     def run(self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict) -> List[Dict[Text, Any]]:
#         # Get user's name and symptoms from the conversation
#         name = tracker.get_slot("name")
#         symptoms = tracker.get_slot("symptoms")

#         # Preprocess symptoms and call the prediction model
#         # Replace this with your actual preprocessing and prediction code
#         predicted_disease = predict_disease(symptoms)

#         # Send the prediction to the user
#         dispatcher.utter_message(
#             template="utter_show_prediction",
#             prediction=predicted_disease
#         )

#         return []




ModuleNotFoundError: No module named 'rasa_sdk'